<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/367_EFIA_Summarization_Utils.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



## LLM Summarization

### Using AI to Explain Insight — Not Create It

This code block defines how **LLMs are used inside the Employee Feedback Intelligence Agent** — and just as importantly, **where they are not used**.

The purpose of this layer is simple and tightly scoped:

> **Translate structured analysis into clear, executive-ready language.**

The LLM never decides priorities, detects patterns, or drives outcomes.
It only explains results that have already been determined by transparent, rule-based logic.

---

## Why Summarization Is a Separate Layer

In many AI systems, analysis and explanation are blended together.
That makes results hard to trust and even harder to audit.

This agent intentionally separates them:

* Rules and statistics determine *what matters*
* LLMs explain *what that means* in natural language

This preserves:

* Accountability
* Explainability
* Executive confidence

---

## Department Summaries

### Turning Local Feedback Into Clear Insight

#### `generate_department_summary`

This function produces a short, executive-ready summary for each department.

It takes as input:

* Actual employee feedback samples
* Precomputed sentiment statistics
* A known department context

The prompt instructs the LLM to:

* Focus on patterns, not anecdotes
* Highlight sentiment direction
* Call out areas needing attention
* Stay concise and actionable

The result is a **2–3 sentence summary** a leader can absorb in seconds.

---

### Built-In Safety: Rule-Based Fallback

If the LLM fails for any reason:

* The system gracefully falls back to a rule-based summary
* No part of the workflow breaks
* No report section is silently missing

This ensures the agent is **operationally reliable**, not brittle.

---

## Theme Summaries

### Explaining Patterns, Not Inventing Them

#### `generate_theme_summary`

Themes are detected earlier using explicit rules and frequency thresholds.
This function simply explains those themes in plain language.

Inputs include:

* Theme name
* Frequency of occurrence
* Real employee examples

The LLM is asked to:

* Describe what employees are saying
* Focus on the core issue or opportunity
* Avoid speculation or recommendation creep

This keeps summaries grounded in evidence.

---

## Executive Summary

### A CEO-Ready Synthesis

#### `generate_executive_summary`

This is the highest-level narrative output of the agent.

It is intentionally constrained:

* Limited to top issues and ideas
* Driven by prior prioritization
* Anchored in concrete statistics

The LLM is guided to:

* Describe the overall state
* Highlight critical risks
* Surface key opportunities
* Offer a high-level directional recommendation

The result is **clarity, not verbosity**.

---

## Strong Guardrails on AI Behavior

Across all summarization functions:

* Temperature is kept low
* Token limits are enforced
* Prompts are narrowly scoped
* Inputs are structured and curated

The LLM never sees:

* Raw, unfiltered data
* Internal scoring logic
* Decision thresholds

This prevents hallucination, drift, and overreach.

---

## Why Leaders Can Trust These Summaries

From a governance perspective, this design ensures:

* AI outputs are explainable
* Conclusions are reproducible
* Summaries align with underlying data
* AI cannot “change the story”

If leadership challenges a summary, the system can always point to:

* The data
* The rules
* The prioritization logic

The LLM is simply the narrator.

---

## Architectural Takeaway

This summarization layer reflects the core philosophy of the agent:

> **The LLM does not decide what the business should do — it explains what the system has already proven.**

By isolating AI to a communication role and backing it with deterministic logic and fallbacks, the agent achieves something rare:

* Natural language insight
* Without sacrificing control

That’s how AI earns a seat in executive decision-making.




# Summarization Utilities for Employee Feedback Intelligence Agent

In [ ]:
"""Summarization Utilities for Employee Feedback Intelligence Agent

LLM-based summarization of feedback themes, departments, and categories.
"""

from typing import List, Dict, Any, Optional
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate


def generate_department_summary(
    department: str,
    feedback_entries: List[Dict[str, Any]],
    sentiment_summary: Dict[str, Any],
    llm_model: str = "gpt-4o-mini",
    temperature: float = 0.3,
    max_tokens: int = 500
) -> str:
    """
    Generate LLM summary for a department's feedback.

    Args:
        department: Department name
        feedback_entries: List of feedback entries for this department
        sentiment_summary: Sentiment summary for this department
        llm_model: LLM model to use
        temperature: LLM temperature
        max_tokens: Max tokens for summary

    Returns:
        Summary text
    """
    # Prepare feedback text samples (up to 10 entries)
    feedback_samples = [e.get("free_text_feedback", "") for e in feedback_entries[:10]]
    feedback_text = "\n".join(f"- {sample}" for sample in feedback_samples)

    # Get sentiment info
    dept_sentiment = sentiment_summary.get("sentiment_by_department", {}).get(department, {})

    prompt = ChatPromptTemplate.from_messages([
        ("system", """You are an organizational intelligence analyst.
        Your job is to create concise, executive-ready summaries of employee feedback.
        Focus on patterns, themes, and actionable insights. Use clear, professional language."""),
        ("human", """Summarize the employee feedback for the {department} department.

Feedback samples:
{feedback_text}

Sentiment breakdown:
- Positive: {positive_count}
- Neutral: {neutral_count}
- Negative: {negative_count}

Create a 2-3 sentence summary that highlights:
1. Main themes or patterns
2. Overall sentiment
3. Key areas needing attention (if any)

Keep it concise and actionable.""")
    ])

    try:
        llm = ChatOpenAI(model=llm_model, temperature=temperature, max_tokens=max_tokens)
        chain = prompt | llm
        response = chain.invoke({
            "department": department,
            "feedback_text": feedback_text,
            "positive_count": dept_sentiment.get("positive", 0),
            "neutral_count": dept_sentiment.get("neutral", 0),
            "negative_count": dept_sentiment.get("negative", 0)
        })
        return response.content.strip()
    except Exception as e:
        # Fallback to rule-based summary
        return (f"{department}: {len(feedback_entries)} feedback entries. "
                f"Sentiment: {dept_sentiment.get('positive', 0)} positive, "
                f"{dept_sentiment.get('negative', 0)} negative, "
                f"{dept_sentiment.get('neutral', 0)} neutral.")


def generate_theme_summary(
    theme: Dict[str, Any],
    feedback_entries: List[Dict[str, Any]],
    llm_model: str = "gpt-4o-mini",
    temperature: float = 0.3,
    max_tokens: int = 300
) -> str:
    """
    Generate LLM summary for a feedback theme.

    Args:
        theme: Theme dictionary
        feedback_entries: List of feedback entries in this theme
        llm_model: LLM model to use
        temperature: LLM temperature
        max_tokens: Max tokens for summary

    Returns:
        Summary text
    """
    theme_name = theme.get("theme_name", "Unknown Theme")
    frequency = theme.get("frequency", 0)
    examples = theme.get("example_feedback", [])

    examples_text = "\n".join(f"- {ex}" for ex in examples[:5])

    prompt = ChatPromptTemplate.from_messages([
        ("system", """You are an organizational intelligence analyst.
        Create concise summaries of recurring feedback themes."""),
        ("human", """Summarize this feedback theme:

Theme: {theme_name}
Frequency: {frequency} occurrences

Example feedback:
{examples_text}

Create a 1-2 sentence summary explaining what employees are saying about this theme.
Focus on the core issue or opportunity.""")
    ])

    try:
        llm = ChatOpenAI(model=llm_model, temperature=temperature, max_tokens=max_tokens)
        chain = prompt | llm
        response = chain.invoke({
            "theme_name": theme_name,
            "frequency": frequency,
            "examples_text": examples_text
        })
        return response.content.strip()
    except Exception as e:
        # Fallback to rule-based summary
        return (f"{theme_name}: {frequency} occurrences. "
                f"Employees report issues/ideas related to {theme_name.lower()}.")


def generate_executive_summary(
    feedback_summary: Dict[str, Any],
    prioritized_issues: List[Dict[str, Any]],
    prioritized_ideas: List[Dict[str, Any]],
    sentiment_summary: Dict[str, Any],
    llm_model: str = "gpt-4o-mini",
    temperature: float = 0.3,
    max_tokens: int = 500
) -> str:
    """
    Generate executive summary of all feedback.

    Args:
        feedback_summary: Overall feedback summary
        prioritized_issues: Top prioritized issues
        prioritized_ideas: Top prioritized ideas
        sentiment_summary: Sentiment summary
        llm_model: LLM model to use
        temperature: LLM temperature
        max_tokens: Max tokens for summary

    Returns:
        Executive summary text
    """
    top_3_issues = [i["feedback_text"][:100] for i in prioritized_issues[:3]]
    top_3_ideas = [i["feedback_text"][:100] for i in prioritized_ideas[:3]]

    prompt = ChatPromptTemplate.from_messages([
        ("system", """You are an executive intelligence analyst.
        Create a concise, high-level summary for senior leadership.
        Focus on patterns, priorities, and actionable insights."""),
        ("human", """Create an executive summary of employee feedback analysis.

Overall Statistics:
- Total feedback: {total_feedback}
- Issues: {total_issues}
- Ideas: {total_ideas}
- Overall sentiment: {overall_sentiment}

Top 3 Priority Issues:
{top_issues}

Top 3 Priority Ideas:
{top_ideas}

Create a 3-4 sentence executive summary that:
1. Highlights the overall state
2. Identifies the most critical issues
3. Notes key opportunities (ideas)
4. Provides a high-level recommendation

Keep it concise and CEO-friendly.""")
    ])

    try:
        llm = ChatOpenAI(model=llm_model, temperature=temperature, max_tokens=max_tokens)
        chain = prompt | llm
        response = chain.invoke({
            "total_feedback": feedback_summary.get("total_feedback", 0),
            "total_issues": feedback_summary.get("total_issues", 0),
            "total_ideas": feedback_summary.get("total_ideas", 0),
            "overall_sentiment": sentiment_summary.get("overall_sentiment", "neutral"),
            "top_issues": "\n".join(f"- {issue}" for issue in top_3_issues),
            "top_ideas": "\n".join(f"- {idea}" for idea in top_3_ideas)
        })
        return response.content.strip()
    except Exception as e:
        # Fallback to rule-based summary
        return (f"Analysis of {feedback_summary.get('total_feedback', 0)} feedback entries: "
                f"{feedback_summary.get('total_issues', 0)} issues and "
                f"{feedback_summary.get('total_ideas', 0)} ideas. "
                f"Overall sentiment: {sentiment_summary.get('overall_sentiment', 'neutral')}. "
                f"Top priority areas identified for leadership attention.")

